Kütüphaneleri ekleme işlemi

### 1. ImageDataGenerator Nedir?

Keras'ın ImageDataGenerator sınıfı, özellikle görüntü veri setleriyle çalışırken, veri ön işleme ve veri artırma (augmentation) işlemleri için kullanılır. Bu sınıf, özellikle büyük veri kümelerinde veriyi anlık olarak (batch halinde) yükler ve aynı zamanda veri çeşitliliğini artırmak için bazı işlemler (döndürme, yakınlaştırma, yatay yansıtma, vb.) uygular.



In [20]:
from keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator oluştur
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)





### 2. Eğitim Verilerinin Yüklenmesi

target_size=(150, 150): Bu parametre, tüm görüntülerin 150x150 piksel boyutuna yeniden ölçeklenmesini sağlar. Modelin sabit boyutlarda girdi alması gerektiği için bu adım önemlidir.

batch_size=32: Bu parametre, modelin her seferinde kaç görüntüyle eğitileceğini belirtir. Yani, veriler her seferinde 32'lik gruplar (batch) halinde modele verilir. Bu, özellikle belleğin verimli kullanılması için önemlidir.

class_mode='categorical': Çoklu sınıf sınıflandırması yapıyorsan bu parametreyi kullanırsın. Görüntüler birden fazla sınıfa (örneğin: buildings, forest, glacier, vb.) ait olabilir, bu nedenle etiketler one-hot encoded bir biçimde verilir. Eğer ikili sınıflandırma (binary classification) olsaydı, class_mode='binary' olurdu.

subset='training': validation_split kullanıldığı için, bu parametre eğitim verilerini belirtir. Yani, doğrulama için ayrılmamış olan %80'lik kısmı eğitim için kullanır.

In [21]:
# Eğitim verilerini yükle
train_generator= train_datagen.flow_from_directory(
    'data\seg_train\seg_train',
    target_size=(150,150), #Veri setindeki her görseli resize etme işlemi
    batch_size=8,# Modele aynı anda kaç görüntü gireceği
    class_mode = "categorical",# Modelde 1 den fazla sınıf olacaksa onun tanımlanması içi categorical
    subset='training'
)

Found 11230 images belonging to 6 classes.


### 2.1 Doğrulama Verilerinin Yüklenmesi

subset='validation': Eğitim verisinin bir kısmı doğrulama için ayrılmıştı. Bu kısım, validation (doğrulama) verilerini yükler. Eğitim sırasında, model her epoch sonunda doğrulama verisi üzerinde test edilir ve sonuçlara göre modelin genel performansı izlenir.


In [22]:
validation_generator = train_datagen.flow_from_directory(
    'data\seg_train\seg_train',
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='validation'
)

Found 2804 images belonging to 6 classes.


In [23]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'data/seg_test/seg_test',
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical'
)


Found 3000 images belonging to 6 classes.


In [24]:
train_datagen.channel_axis

3

In [25]:
train_generator.class_indices

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

In [26]:
train_generator.image_shape

(150, 150, 3)

Model Mimarisi Oluşturma

In [27]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense

In [28]:
model = Sequential()

model.add(Conv2D(filters=16,kernel_size=2,padding="same",strides=1,activation="relu",input_shape=(150,150,3)))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,kernel_size=2,padding="same",strides=1,activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=2,padding="same",strides=1,activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(512,activation="relu"))

model.add(Dropout(0.4))

model.add(Dense(6,activation="softmax"))

model.summary()




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 150, 150, 16)      208       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 75, 75, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 75, 75, 32)        2080      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 37, 37, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 37, 37, 64)        8256      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 18, 18, 64)       

In [31]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
 metrics=['accuracy'])

In [33]:
train_generator.samples

11230

In [34]:
from keras.callbacks import ModelCheckpoint


checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1,
 save_best_only=True)
# Modeli eğitme (fit)


hist = model.fit(
    train_generator,  # Eğitim verisi generator
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Eğitim için gereken adım sayısı
    validation_data=validation_generator,  # Doğrulama verisi generator
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # Doğrulama için gereken adım sayısı
    epochs=100,  # Kaç epoch eğitim yapılacağı
    callbacks=[checkpointer],  # En iyi modeli kaydetme callback'i
    verbose=2,  # Eğitim sırasında daha az detaylı çıktı almak için 2 (ya da daha fazla bilgi için 1)
    shuffle=True  # Eğitim verilerini karıştırma
)

8

train_generator ve validation_generator:

-->train_generator ve validation_generator dizinlerden veriyi yükleyen flow_from_directory fonksiyonu ile oluşturuldu. Model, bu generator'lardan veriyi çekerek eğitim yapacak.
steps_per_epoch:

-->Bu parametre, bir epoch sırasında kaç adımda eğitim verisinin işleneceğini belirtir. Her batch bir adım olduğundan, steps_per_epoch değeri train_generator.samples // train_generator.batch_size olarak ayarlandı. Bu, tüm verinin bir epoch'ta işlenmesini sağlar.
    validation_steps:

-->Doğrulama verisi için kaç adımda işlemin tamamlanacağını belirtir. Aynı mantıkla, validation_steps, validation_generator.samples // validation_generator.batch_size olarak hesaplanır.
    epochs:

-->Modelin kaç epoch boyunca eğitileceğini belirler. Bu örnekte 100 epoch seçildi.
    callbacks:

-->Model eğitilirken, en iyi doğrulama başarımına sahip olan modelin ağırlıklarının (model.weights.best.hdf5) kaydedilmesi için ModelCheckpoint callback kullanılıyor. Bu, overfitting'den kaçınmak ve en iyi modeli almak için yararlıdır.
Bu şekilde, dizinlerde organize edilen verilerle modelini eğitebilirsin.

In [38]:
from keras.callbacks import ModelCheckpoint


checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1,
 save_best_only=True)
# Modeli eğitme (fit)


hist = model.fit(
    train_generator,  # Eğitim verisi generator
    validation_data=validation_generator,  # Doğrulama verisi generator
    epochs=100,  # Kaç epoch eğitim yapılacağı
    callbacks=[checkpointer],  # En iyi modeli kaydetme callback'i
    verbose=2,  # Eğitim sırasında daha az detaylı çıktı almak için 2 (ya da daha fazla bilgi için 1)
    shuffle=True  # Eğitim verilerini karıştırma
)

Epoch 1/100
